In [1]:
import os
os.chdir('../')
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [11]:
from sklearn.ensemble import *
from sklearn.metrics import *
from sklearn.svm import *
from sklearn.linear_model import *
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.ensemble import *
from sklearn.gaussian_process import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.neural_network import *
from sklearn.tree import *
import numpy as np
from sklearn.impute import *
from xgboost import XGBClassifier
from sklearn.feature_selection import *
import time

In [3]:
data = pd.read_csv('data/UNSW_NB15_training-set.csv',index_col='id')
data.head()

dur proto service state  spkts  dpkts  sbytes  dbytes       rate  \
id                                                                          
1   0.121478   tcp       -   FIN      6      4     258     172  74.087490   
2   0.649902   tcp       -   FIN     14     38     734   42014  78.473372   
3   1.623129   tcp       -   FIN      8     16     364   13186  14.170161   
4   1.681642   tcp     ftp   FIN     12     12     628     770  13.677108   
5   0.449454   tcp       -   FIN     10      6     534     268  33.373826   

    sttl  dttl         sload          dload  sloss  dloss      sinpkt  \
id                                                                      
1    252   254  14158.942380    8495.365234      0      0   24.295600   
2     62   252   8395.112305  503571.312500      2     17   49.915000   
3     62   252   1572.271851   60929.230470      1      6  231.875571   
4     62   252   2740.178955    3358.622070      1      3  152.876547   
5    254   252   8561.499023    3987.059814      2      1   47.750333   

        dinpkt          sjit          djit  swin       stcpb       dtcpb  \
id                                                                         
1     8.375000     30.177547     11.830604   255   621772692  2202533631   
2    15.432865     61.426934   1387.778330   255  1417884146  3077387971   
3   102.737203  17179.586860  11420.926230   255  2116150707  2963114973   
4    90.235726    259.080172   4991.784669   255  1107119177  1047442890   
5    75.659602   2415.837634    115.807000   255  2436137549  1977154190   

    dwin    tcprtt    synack    ackdat  smean  dmean  trans_depth  \
id                                                                  
1    255  0.000000  0.000000  0.000000     43     43            0   
2    255  0.000000  0.000000  0.000000     52   1106            0   
3    255  0.111897  0.061458  0.050439     46    824            0   
4    255  0.000000  0.000000  0.000000     52     64            0   
5    255  0.128381  0.071147  0.057234     53     45            0   

    response_body_len  ct_srv_src  ct_state_ttl  ct_dst_ltm  ct_src_dport_ltm  \
id                                                                              
1                   0           1             0           1                 1   
2                   0          43             1           1                 1   
3                   0           7             1           2                 1   
4                   0           1             1           2                 1   
5                   0          43             1           2                 2   

    ct_dst_sport_ltm  ct_dst_src_ltm  is_ftp_login  ct_ftp_cmd  \
id                                                               
1                  1               1             0           0   
2                  1               2             0           0   
3                  1               3             0           0   
4                  1               3             1           1   
5                  1              40             0           0   

    ct_flw_http_mthd  ct_src_ltm  ct_srv_dst  is_sm_ips_ports attack_cat  \
id                                                                         
1                  0           1           1                0     Normal   
2                  0           1           6                0     Normal   
3                  0           2           6                0     Normal   
4                  0           2           1                0     Normal   
5                  0           2          39                0     Normal   

    label  
id         
1       0  
2       0  
3       0  
4       0  
5       0

In [4]:
def preprocess_initial(df):
       
    df       = df.drop(['attack_cat','proto','service','state'], axis=1)
    
    right = ['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', "rate",  "dttl",
           'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit',
            'djit',"swin", 'stcpb']
    
    left = ["sttl"]
    
    for i in left:
        df[i] = df[i]**3
        
    for i in right:
        df[i] = (df[i]+1).transform(np.log)
    return df

In [5]:
def preprocess_final(X_train, X_test, y_train, y_test):
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test

In [6]:
def f_score(X_train, X_test, y_train, y_test):
    
    classifiers = [
        KNeighborsClassifier(n_neighbors=10),
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        GradientBoostingClassifier(),
        AdaBoostClassifier(),
        BaggingClassifier(),
        ExtraTreesClassifier(),
        MLPClassifier(max_iter=500),
        BernoulliNB(),
        PassiveAggressiveClassifier(),
        RidgeClassifierCV(),
        SGDClassifier(),
        LogisticRegression(max_iter=500),
    ]
    
    model = XGBClassifier(silent=True,
                      booster = 'gbtree',
                      objective='binary:logistic',
                      n_estimators=500, 
                      max_depth=10,
                      verbosity=0
                     )
    
    eval_set = [(X_test, y_test)]
    eval_metric = ["logloss"]
    
    model.fit(X_train, y_train,
              early_stopping_rounds=50, 
              eval_metric=eval_metric, 
              eval_set=eval_set)
    
    yxgb=model.predict(X_test)
    f = f1_score(y_true=y_test,y_pred=yxgb,average="weighted")
    
    yield 'XGBoost',f
    
    for clf in classifiers:
        clf.fit(X_train,y_train.ravel())
        y_pred = clf.predict(X_test)
        f = f1_score(y_true=y_test,y_pred=y_pred,average="weighted")
        
        yield clf.__class__.__name__ , round(f,4)

In [7]:
def category(X,y):
    
    d = []
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    fold = 0
    for train_ix, test_ix in kfold.split(X, y):
        fold+=1
        
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]

        X_train, X_test, y_train, y_test = preprocess_final(X_train, X_test, y_train, y_test)

        l=[]
        for class_name , score in f_score(X_train, X_test, y_train, y_test):
            l.append({'name':class_name,f"fold{fold}" : score})
            
        if len(d)==0:
            d = pd.DataFrame(l).set_index('name')
        else:
            d = pd.concat([d,pd.DataFrame(l).set_index('name')],axis=1)
        print(f"Completed Fold{fold}")
    return d

In [8]:
df_final = preprocess_initial(data)
train_y  = df_final['label']
train_x  = df_final.drop(['label'],axis=1)

result = category(train_x.values,train_y.values)
result['mean'] = result.mean(axis=1)
result

[0]	validation_0-logloss:0.47505
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.35333
[2]	validation_0-logloss:0.27683
[3]	validation_0-logloss:0.22613
[4]	validation_0-logloss:0.19041
[5]	validation_0-logloss:0.16502
[6]	validation_0-logloss:0.14700
[7]	validation_0-logloss:0.13317
[8]	validation_0-logloss:0.12356
[9]	validation_0-logloss:0.11605
[10]	validation_0-logloss:0.11075
[11]	validation_0-logloss:0.10674
[12]	validation_0-logloss:0.10362
[13]	validation_0-logloss:0.10107
[14]	validation_0-logloss:0.09947
[15]	validation_0-logloss:0.09760
[16]	validation_0-logloss:0.09651
[17]	validation_0-logloss:0.09524
[18]	validation_0-logloss:0.09393
[19]	validation_0-logloss:0.09327
[20]	validation_0-logloss:0.09273
[21]	validation_0-logloss:0.09244
[22]	validation_0-logloss:0.09210
[23]	validation_0-logloss:0.09191
[24]	validation_0-logloss:0.09159
[25]	validation_0-logloss:0.09155
[26]	validation_0-logloss:0.09139
[27]	validation_0-loglos

fold1     fold2    fold3     fold4     fold5  \
name                                                                           
XGBoost                      0.962596  0.960028  0.96302  0.961355  0.961716   
KNeighborsClassifier         0.945100  0.942300  0.94680  0.942400  0.943800   
DecisionTreeClassifier       0.948800  0.947300  0.94940  0.951000  0.948400   
RandomForestClassifier       0.960100  0.958900  0.96120  0.959700  0.961100   
GradientBoostingClassifier   0.945000  0.942100  0.94490  0.942100  0.945300   
AdaBoostClassifier           0.943200  0.935300  0.93830  0.937800  0.941000   
BaggingClassifier            0.955400  0.955400  0.95660  0.958600  0.957300   
ExtraTreesClassifier         0.960000  0.958800  0.95970  0.960100  0.960500   
MLPClassifier                0.948500  0.946600  0.94920  0.946500  0.948600   
BernoulliNB                  0.721500  0.725900  0.72770  0.720300  0.727900   
PassiveAggressiveClassifier  0.906200  0.906100  0.77780  0.913300  0.913200   
RidgeClassifierCV            0.922800  0.922200  0.92530  0.923400  0.921800   
SGDClassifier                0.923800  0.921900  0.92670  0.924200  0.922600   
LogisticRegression           0.921300  0.920700  0.92540  0.921000  0.922300   

                                fold6     fold7     fold8     fold9    fold10  \
name                                                                            
XGBoost                      0.964327  0.962247  0.965515  0.960967  0.959967   
KNeighborsClassifier         0.945600  0.942500  0.945900  0.944300  0.942100   
DecisionTreeClassifier       0.952400  0.950400  0.952500  0.949200  0.946900   
RandomForestClassifier       0.962300  0.960200  0.963400  0.958400  0.959200   
GradientBoostingClassifier   0.946400  0.944900  0.947800  0.943400  0.945700   
AdaBoostClassifier           0.936900  0.937400  0.943900  0.938500  0.939100   
BaggingClassifier            0.959300  0.956400  0.959300  0.958100  0.954300   
ExtraTreesClassifier         0.961600  0.959200  0.962100  0.958000  0.958000   
MLPClassifier                0.947700  0.948800  0.952300  0.947700  0.948000   
BernoulliNB                  0.726700  0.724000  0.727200  0.722200  0.720600   
PassiveAggressiveClassifier  0.920000  0.906600  0.909100  0.867600  0.834800   
RidgeClassifierCV            0.926000  0.924600  0.926300  0.925400  0.923800   
SGDClassifier                0.926600  0.926100  0.927100  0.926000  0.925800   
LogisticRegression           0.926500  0.926000  0.925700  0.925200  0.924700   

                                 mean  
name                                   
XGBoost                      0.962174  
KNeighborsClassifier         0.944080  
DecisionTreeClassifier       0.949630  
RandomForestClassifier       0.960450  
GradientBoostingClassifier   0.944760  
AdaBoostClassifier           0.939140  
BaggingClassifier            0.957070  
ExtraTreesClassifier         0.959800  
MLPClassifier                0.948390  
BernoulliNB                  0.724400  
PassiveAggressiveClassifier  0.885470  
RidgeClassifierCV            0.924160  
SGDClassifier                0.925080  
LogisticRegression           0.923880